# 멀티헤드 어텐션과 피드포워드

In [ ]:
# self-attention에서 만든 Head class를 가져옴

class MultiheadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)]) # ModuleList의 기능?

    def forward(self, inputs):
        return torch.cat([head(inputs) for head in self.heads], dim=-1)

# 피드포워드

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embd),
            nn.Dropout(dropout),
        )
    
    def forward(self, input_tensor):
        return self.layer(input_tensor)

# Blocks 만들기

In [ ]:
# Attention, FeedForward 기능 모두 하나로 합쳐 돌아가는 트랜스포머 기본구조

class Block(nn.Module):
    def __init__(self, n_embed, n_heads):
        super().__init__()
        head_size = n_embed // n_heads
        self.attention = MultiheadAttention(n_heads, head_size)
        self.feedforward = FeedForward(n_embed)
        self.layer_norm1 = nn.LayerNorm(n_embed)
        self.layer_norm2 = nn.LayerNorm(n_embed)

    def forward(self, input_tensor):
        # 여기서의 입력의 shape는 n_embed 뿐인가? (왜 self.layer_norm1에 input_tensor 통째로 넣지?)
        input_tensor = input_tensor + self.attention(self.layer_norm1(input_tensor))
        input_tensor = input_tensor + self.feedforward(self.layer_norm2(input_tensor))
        return input_tensor


# 토크나이저 만들기

In [ ]:
import os
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from datasets import load_dataset
from transformers import PreTrainedTokenizerFast

# 저장 경로
SAVE_DIR = "/content"

# 디렉터리가 없다면 생성
os.mkdir(SAVE_DIR, exist_ok=True)

VOCAB_SIZE = 10000

# 토크나이저 초기화
tokenizer = Tokenizer(BPE(unk_token="<unk>"))
tokenizer.pre_tokenizers = Whitespace()

# 트레이너 준비 (vocab_size 지정)
trainer = BpeTrainer(
    special_tokens=["<unk>", "<s>", "</s>", "<pad>"],
    vocab_size=VOCAB_SIZE
)

# 토크나이저 학습
def batch_iteration(batch_size=1000):
    for i in range(0, len(dataset["train"]), batch_size):
        yield dataset["train"][i : i + batch_size]["document"]

tokenizer.train_from_iterator(batch_iteration(), trainer=trainer)


In [ ]:
# 토크나이저를 JSON 파일로 저장
tokenizer_path = os.path.join(SAVE_DIR, "tokenizer.json")
tokenizer.save(tokenizer_path)

# 토크나이저를 Hugging Face 형식으로 변환
huggingface_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    unk_token="<unk>",
    bos_token="<s>",
    eos_token="</s>",
    pad_token="<pad>"
)

# Hugging Face 형식의 토크나이저 저장
huggingface_path = os.path.join(SAVE_DIR, "huggingface_tokenizer")
huggingface_tokenizer.save_pretrained(huggingface_path)

In [ ]:
# Hugging Face 형식의 토크나이저 로드
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(huggingface_path)

# 어휘 크기 확인
print(f"Vocabulary size : {len(tokenizer.get_vocab())}")

# 테스트
test_texts = ["안녕하세요", "자연어 처리는 매우 흥미로운 분야입니다", "인공지능과
기계학습의 발전이 놀랍습니다"]
for text in test_texts:
    encoded = tokenizer.encode(text) # 텍스트 토크나이징하여 숫자로 변환
    print(f"Original: {text}")
    print(f"Encoded: {encoded}")
    print(f"Decoded: {tokenizer.decode(encoded)}")
    print(f"Tokens: {tokenizer.convert_ids_to_tokens(encoded)}")
    print()